# SISTEMA DE RECOMENDACIÓN BASADO EN CONTENIDO

Comenzamos importando las librerias necesarias y descargando ciertos archivos con ayuda de nltk

In [2]:
import pandas as pd 
import numpy as np
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

from itertools import combinations

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ronaldo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ronaldo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Los datos utilizados provienen del dataset de MovieLens, ligeramente modificado. Originalmente no contenía los argumentos, los cuales han sido añadidos. 

In [3]:
path = r".\archivos\MovieLens_con_argumento.csv"
movies = pd.read_csv(path, sep=',')
movies.head()

,movieId,title,genres,argumento
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji (1995),Adventure|Children|Fantasy,When two kids find and play a magical board ga...
2,3,Grumpier Old Men (1995),Comedy|Romance,John and Max resolve to save their beloved bai...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Based on Terry McMillan's novel, this film fol..."
4,5,Father of the Bride Part II (1995),Comedy,George Banks must deal not only with the pregn...


Cada película contiene un ```movieId``` que no corresponde con el índice del DataFrame, por lo que se crean dos diccionarios con la relación que guardan entre el ```index``` y el ```movieId```

In [4]:
# Tablas de referencia de los movield-index, cuyo objetivo es ahorrar líneas de código
dict_index_movieid = movies['movieId'].to_dict()
dict_movieid_index = {value: key for key, value in dict_index_movieid.items()}

Para la realización de un sistema de recomendación basado en contenido es necesario estraer información de cada elemento (película), en este caso se usarán los argumentos y los géneros a los que pertenece para la búsqueda de otras películas similares.

La técnica empleada para esta tarea será **Bag-Of-Words**, convirtiendo el texto a un vector algebraico que lo represente utilizando **TF-IDF** para el cálculo de la relevancia de cada palabra dentro del cada argumento del DataFrame. 

Los principales problemas que representa la técnica empleada son:
    
- Existencia de palabras muy repetitivas y que no aportan significado al texto (conectores, artículos, etc)

- Dificultades a la hora de comparar textos con una diferencia en extensión alta

Con respecto al primer problema, usaremos la función ```stopwords``` que contiene los principales palabras de uso común y que no aportan significado al texto analizado, además se eliminarán los signos de puntuación. Como medida adicional también se procederá a la reducción de las palabras a sus raíces usando la función ```SnowballStemmer``` para una mejor comparación de las palabras.

La extensión de los textos a analizar es parecida por lo que podemos ignorar el segundo problema.

Para la realización del vector se usará la función de Scikit-Learn ```CountVectorizer``` que automatiza el proceso de contar las palabras y a la cual podemos darle como argumento de entrada una función de preprocesamiento, es decir, una funciónn que se le aplicará a cada texto del DataFrame para preparar los datos. Además para dotar de mayor significado modificaremos el parámetros ```min_df``` que expresa el número mínimo de argumentos distintos en los que debe aparecer una palabra para ser tenida en cuenta. 


In [5]:
def preprocess (argument):

    # Transformar todo el texto en letra minúscula y eliminar los valores numéricos
    s = argument.lower()
    s = re.sub(r"\d+", "", s)

    # Eliminar signos de puntuación y palabras sin significado para el análisis
    stop_word = stopwords.words('english')
    words = word_tokenize(s)
    words = [x for x in words if x not in stop_word + list(string.punctuation)]

    # Reducir las palabras a su raiz
    stemmer = SnowballStemmer('english')
    roots = []
    for word in words:
        roots.append(stemmer.stem(word))
    
    # Concatenar la lista de palabras para el CountVectorizer
    s = ' '.join(roots)

    return s

In [6]:
count_arguments = CountVectorizer(preprocessor=preprocess, min_df=5)

arguments_bow = (count_arguments
                 .fit_transform(movies['argumento'])
                 .toarray())

Para una mejor visualización y manipulación de los datos el *array* generado se tranformará en un *DataFrame* con las palabras ordenadas alfabéticamente

In [7]:
columns_arguments = [tup[0] for tup in
                     sorted(count_arguments.vocabulary_.items(),
                            key=lambda x: x[1])]

arguments_bow_df = pd.DataFrame(arguments_bow,
                                columns = columns_arguments,
                                index = movies['title'])

arguments_bow_df.head()

,abandon,abduct,abil,abl,aboard,aborigin,abort,abroad,abrupt,absenc,...,youngster,youth,yuppi,zealand,zero,zeus,zombi,zone,zoo,zoologist
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jumanji (1995),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Grumpier Old Men (1995),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Waiting to Exhale (1995),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Father of the Bride Part II (1995),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Como ya mencionado anteriormente, también se incluirán los géneros dentro del Bag-Of-Words. Para ello debemos realizar diferentes combinaciones 2 a 2 (o de uno en uno) de los géneros a los que pertenece cada película. 

Los géneros aparecen se presentan como un string del tipo: ```Animation|Children|Comedy|Fantasy``` por lo que debemos prepararlo para realizar las combinaciones.

In [8]:
def tokenizer_genres(string_genres):
    genres = string_genres.split('|')
    answer = []
    for size in [1,2]:
        combs = ['Género - ' + '|'.join(sorted(tup))
                 for tup in combinations(genres, r=size)]
        answer = answer + combs
    return sorted(answer)

In [9]:
tokenizer_genres('Animation|Children|Comedy|Fantasy')

['Género - Animation',
 'Género - Animation|Children',
 'Género - Animation|Comedy',
 'Género - Animation|Fantasy',
 'Género - Children',
 'Género - Children|Comedy',
 'Género - Children|Fantasy',
 'Género - Comedy',
 'Género - Comedy|Fantasy',
 'Género - Fantasy']

Ahora se realizará el conteo, pero en este caso no necesitamos procesar el texto, solo que cuente las combinaciones de géneros comos si de palabras se tratase.

In [10]:
count_genres = CountVectorizer(tokenizer = tokenizer_genres,
                               token_pattern = None,
                               lowercase = False)

count_genres.fit(movies['genres'])
genres_bow = count_genres.fit_transform(movies['genres']).toarray()

columns_genres = [tup[0] for tup in
                  sorted(count_genres.vocabulary_.items(),
                         key = lambda x: x[1])]

genres_bow_df = pd.DataFrame(genres_bow,
                             columns = columns_genres,
                             index = movies['title'])

In [11]:
arg_gen_bow = np.hstack((arguments_bow, genres_bow))

arg_gen_bow_df = pd.DataFrame(arg_gen_bow,
                              columns = columns_arguments + columns_genres,
                              index = movies["title"])

Se ha conseguido un *DataFrame* con el Bag-Of-Words de los argumentos y los géneros. Los vectores solo contiene el el número de repeticiones de cada palabra para cada película, realmente esto no representa el peso real de la palabra por lo que se recurre al cálculo de los vectores **TF-IDF**.

En Python, teniendo un *DataFrame* preparado, como el que se tiene, se puede hacer uso de la función ```TfidfTransformer``` de la libreria *Scikit-Learn* para el cálculo de los vectores TF-IDF facilmente.

In [12]:
tf_idf = TfidfTransformer()
tf_idf_movies = tf_idf.fit_transform(arg_gen_bow_df).toarray()

tf_idf_movies_df = pd.DataFrame(tf_idf_movies,
                                columns = columns_arguments + columns_genres,
                                index = movies["title"])

Calculados los vectores TF-IDF se procede al calculo de similaridad entre éstos.

Para calcular la similaridad entre vectores (los vectores TF-IDF que representan a cada una de las películas) se pueden utilizar varias técnicas: distancia euclídea, correlación de Pearson... Pero para vectores TF-IDF la ```cosine similarity``` suele ser la más utilizada, ya que funciona bien en vectores de muchas dimensiones, como los que se tienen.

La cosine similarity define la similaridad en base al coseno del ángulo entre los vectores de las películas. La fórmula del cosine similarity para dos vectores **a** y **b** es:

$$
cosine \, sim_{(\textbf{a},\textbf{b})} = cos\, \theta_{(\textbf{a},\textbf{b})} = \frac{\textbf{a} \cdot \textbf{b}}{\Vert \textbf{a} \Vert \, \Vert \textbf{b}\Vert} = \frac{\sum_{i=1}^n a_i b_i}{\sqrt{\sum_{i=1}^n a_i^2} \, \sqrt{\sum_{i=1}^n b_i^2}}
$$

Donde:

- $n$ es el número de coordenadas/dimensiones de nuestros vectores **a** y **b**
- $a_i$ y $b_i$ con coordenadas $i$ de los vectores **a** y **b** respectivamente

Puesto que tenemos muchas películas y queremos calcular la similaridad entre todas ellas dos a dos, podemos crear una matriz de cosine similarities con la función ```cosine_similarity``` de Scikit-Learn:



In [13]:
cosine_sims = cosine_similarity(tf_idf_movies_df)
matrix_sims_df = pd.DataFrame(cosine_sims,
                              index = movies["title"],
                              columns = movies["title"])

Esta matriz/DataFrame tiene las mismas filas que columnas, y cada celda de la misma es la cosine similarity entre la película especificada en la fila y la película especificada en la columna.

Puesto que la cosine similarity siempre está entre 0 y 1 (donde 0 es similaridad nula y 1 es máxima), una película consigo misma siempre tiene cosine similarity de 1.

En cada fila del DataFrame anterior podemos ver la cosine similarity entre una determinada película con todas las demás

Se sustituirán las cosine similarities de cada película consigo misma por NaN; para  que no se tengan en cuenta:

In [14]:
np.fill_diagonal(matrix_sims_df.values, np.nan)
matrix_sims_df.head()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),NaN,0.142501,0.023003,0.006340,0.006258,0.000000,0.005630,0.076392,0.000000,0.011483,...,0.056961,0.022768,0.007022,0.019441,0.000000,0.134400,0.170625,0.000000,0.021156,0.004312
Jumanji (1995),0.142501,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.127335,0.103491,0.044095,...,0.017364,0.000000,0.021923,0.000000,0.055704,0.077386,0.041150,0.000000,0.000000,0.000000
Grumpier Old Men (1995),0.023003,0.000000,NaN,0.039387,0.006378,0.000000,0.034972,0.000000,0.000000,0.000000,...,0.005669,0.000000,0.007158,0.000000,0.000000,0.005396,0.026032,0.000000,0.000000,0.123431
Waiting to Exhale (1995),0.006340,0.000000,0.039387,NaN,0.007548,0.000000,0.041385,0.000000,0.000000,0.044127,...,0.006709,0.006286,0.039319,0.036984,0.000000,0.006385,0.131024,0.007376,0.000000,0.005201
Father of the Bride Part II (1995),0.006258,0.000000,0.006378,0.007548,NaN,0.060354,0.006702,0.000000,0.000000,0.000000,...,0.006622,0.029766,0.083754,0.000000,0.000000,0.006302,0.007082,0.064036,0.000000,0.005134


Ya se puede, por tanto, buscar las películas más similares a cualquiera del dataset de películas. Simplemente se ha de buscar en la fila correspondiente a la película de la que se quiera encontrar las más similares; buscando las **$K$** películas con mayor cosine similarity con ella.

Para acelerar los cálculos, se generarán dos *arrays*:

- ```order_movies_csim_row```: Array donde cada fila representa los indices de las peliculas ordenadas segun la similitud que tienen con la pelicula elegida.

- ```order_csim_row```: Array donde cada fila representa la cosine similarity de las peliculas ordenadas de mayor a menor similitud.

In [15]:
order_movies_csim_row = np.argsort(-cosine_sims, axis=1)
order_movies_csim_row[0]

array([2998, 2353, 7351, ..., 3826, 7020,    0], dtype=int64)

In [16]:
order_csim_row = -np.sort(-cosine_sims, axis=1)
order_csim_row[0]

array([0.40397622, 0.39966961, 0.3888024 , ..., 0.        , 0.        ,
              nan])

Con estas dos matrices ```(order_movies_csim_row y order_csim_row)``` podemos ya calcular muy fácilmente qué películas son las más similares a una cualquiera, y cuáles son los valores de dichas similaridades.

Para hacerlo más fácil, vamos a crear una función que tome como argumentos un movieId y un número $k$, y devuelva el listado de las top $k$ películas más similares a ella:

In [17]:
def top_k_sim(movieId, k):
    """
    Devuelve un DataFrame con el top k
    de películas más similares a la
    introducida en el argumento movieId.
    """
    
    # Conversión del movieId a el número de fila
    row_csim = dict_movieid_index[movieId]

    # Selección de valores en las dos matrices y cogemos los top k
    top_k = order_movies_csim_row[row_csim][:k]
    csim_top_k = order_csim_row[row_csim][:k]
    
    # Preparamos la salida de la función, cogiendo
    # el dataset original, quedándonos solo con las
    # películas del top k, y añadimos una columna
    # con las similaridades:
    top_k_df = movies.iloc[top_k].copy()
    top_k_df["similaridad"] = csim_top_k
    
    return top_k_df

In [18]:
top_k_sim(1, k=6)

,movieId,title,genres,argumento,similaridad
2998,4016,"Emperor's New Groove, The (2000)",Adventure|Animation|Children|Comedy|Fantasy,Emperor Kuzco is turned into a llama by his ex...,0.403976
2353,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,"When Woody is stolen by a toy collector, Buzz ...",0.399670
7351,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,The toys are mistakenly delivered to a day-car...,0.388802
5084,8015,"Phantom Tollbooth, The (1970)",Adventure|Animation|Children|Fantasy,Milo is a boy who is bored with life. One day ...,0.386209
8921,136016,The Good Dinosaur (2015),Adventure|Animation|Children|Comedy|Fantasy,In a world where dinosaurs and humans live sid...,0.383690
6445,51939,TMNT (Teenage Mutant Ninja Turtles) (2007),Action|Adventure|Animation|Children|Comedy|Fan...,When the world is threatened by an ancient evi...,0.383128


Se puede ir un paso más allá, el recomendador anterior no es un sistema de recomendación personalizado, por lo que no tiene en cuenta las preferencias de un usuario. Se limita a mostrar películas similares a una elegida, y éstas siempre son las mismas para cualquier usuario y no tiene en cuenta las preferencias personales.

Se puede convertir sencillamente el recomendador anterior en un sistema de recomendación personalizado:

> Si tenemos un listado de los ratings (de 1-5) que ha dado un usuario a varias películas, podemos recomendar películas que mejor combinen el contenido que le gusta a dicho usuario

La manera más útil de realizar ésto es estimando el *rating* que le daría el usuario a todas las peliculas del catálogo, y ordenándolas de mayor a menor.

La fórmula más utilizada en la literatura sobre sistemas de recomendación para estimar el rating $\hat{r}$ que le daría un usuario $u$ a una película $pel$ podemos representarlo como $\hat{r}_{u,pel}$, y podemos calcularlo como:

$$
\hat{r}_{u,pel} = \frac{\sum_{otrapel \in VeRat} sim_{(pel, otrapel)} \times r_{u, otrapel}}{\sum_{otrapel \in VeRat} sim_{(pel, otrapel)}}
$$

Donde:

- $VeRat$ es la lista de las $K$ peliculas más similares a $pel$ y que han sido *rateadas* (puntuadas) por el usuario $u$

- $r_{(u, otrapel)}$ es el rating que nuestro usuario $u$ le ha dado a la película $otrapel$; que es cada una de las que pertenece a $VeRat$

- $sim_{(pel, otrapel)}$ es la cosine similarity entre $pel$ y $otrapel$

Así que básicamente consiste en ponderar la puntuación de las películas más similares (vecinas) a la que queremos estimar, en base a cómo de similar es cada vecina a la que buscamos.

Para realizar y testear el nuevo recomendador se creará un un rating ficticio de un usuario llamado rex.

In [20]:
rex = pd.DataFrame({'movieId': [1, 106696, 1240, 1580, 597],
                    'rating': [5, 3, 5, 4, 1]})

view_rex = rex.merge(movies,
                     how = 'inner',
                     on = 'movieId')

view_rex

,movieId,rating,title,genres,argumento
0,1,5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
1,106696,3,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,When the newly crowned Queen Elsa accidentally...
2,1240,5,"Terminator, The (1984)",Action|Sci-Fi|Thriller,A human soldier is sent from 2029 to 1984 to s...
3,1580,4,Men in Black (a.k.a. MIB) (1997),Action|Comedy|Sci-Fi,A police officer joins a secret organization t...
4,597,1,Pretty Woman (1990),Comedy|Romance,A man in a legal but hurtful business needs an...
